In [ ]:
!pip install -q openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.6 MB/s eta 0:00:00


In [ ]:
# importing openai module into your openai environment
import openai

# assigning API KEY to initialize openai environment
openai.api_key = ''


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

with open('/content/drive/MyDrive/train_v9.json', 'r', encoding = 'utf-8-sig') as f:
  train_list = json.load(f)

with open('/content/drive/MyDrive/dev_v9.json', 'r', encoding = 'utf-8-sig') as f:
  dev_list = json.load(f)

total_data = train_list + dev_list

In [ ]:
non_referral = []
referral = []

str1 = 'سوره'
str2 = 'نام ببرید'
str3 = 'یکی از انواع'

for qa in total_data:
  question = qa['question']
  if str1 not in question and str2 not in question and str3 not in question:
    non_referral.append(qa)
  else:
    referral.append(qa)

#referential

In [ ]:
# model="gpt-4o-instruct"
model = "gpt-3.5-turbo-instruct"

new_list = []

for qa in referral:
  prompt = 'پاسخ پرسش'+ '"' + qa['question'] + '"' + 'بر اساس متن ترجمه مکارم شیرازی از آیات قرآن چیست؟ فقط پاسخ را بدون جمله کامل ارائه دهید'
  # print(prompt)
  response = openai.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=50,
        n=1
  )
  qa['gpt'] = response.choices[0].text
  new_list.append(qa)
  # print(response)
  # break

In [ ]:
with open('/content/drive/MyDrive/chatgpt.json', 'w', encoding = 'utf-8-sig') as f:
  json.dump(new_list, f, ensure_ascii = False)


# TEST CHATGPT

In [ ]:
!pip install -q evaluate
!pip install -q parsivar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 57.0 MB/s eta 0:00:00


In [ ]:
from evaluate import load
import json
from parsivar import Normalizer

my_normalizer = Normalizer()

with open('/content/drive/MyDrive/chatgpt.json', 'r', encoding = 'utf-8-sig') as f:
  data_list = json.load(f)

predictions = []
references = []

for data in data_list:
  if data['answers'][0]['text'] in data['gpt']:
    # predictions.append(data['answers'][0]['text'])
    # references.append(data['answers'][0]['text'])
    predictions.append(my_normalizer.normalize(data['answers'][0]['text']))
    references.append(my_normalizer.normalize(data['answers'][0]['text']))
  else:
    # predictions.append(data['gpt'].strip())
    # references.append(data['answers'][0]['text'])
    predictions.append(my_normalizer.normalize(data['gpt'].strip()))
    references.append(my_normalizer.normalize(data['answers'][0]['text']))

exact_match_metric = load("exact_match")
results = exact_match_metric.compute(predictions=predictions, references=references)

In [ ]:
results

{'exact_match': 0.06958561376075059}

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

# f1 = f1_score(ground_truth, predicted_output, average='micro')
f1 = f1_score(references, predictions, average='weighted')

f1

0.10428085820434199

# non_referential
### gpt-4o

In [ ]:
model="gpt-4o"
# model = "gpt-3.5-turbo-instruct"

new_list = []

for qa in referral:
  prompt = 'پاسخ پرسش'+ '"' + qa['question'] + '"' + 'بر اساس متن ترجمه مکارم شیرازی از آیات قرآن چیست؟ فقط پاسخ را بدون جمله کامل ارائه دهید'
  # print(prompt)

  response = openai.ChatCompletion.create(
        model=model,
        messages = [
        {"role": "user", "content": prompt}
        ],
        max_tokens=50,
        n=1
  )
  qa['gpt'] = response.choices[0].message['content']
  new_list.append(qa)
  # print(response)
  # break


### gpt-3.5

In [ ]:
len(non_referral)

5078

In [ ]:
# model="gpt-4o-instruct"
model = "gpt-3.5-turbo-instruct"

new_list2 = []

for qa in non_referral[2500:]:
  prompt = ' طبق آیات قرآن' + qa['question'] + ' پاسخ را به‌صورت کوتاه و بدون جمله کامل از ترجمه مکارم شیرازی استخراج کنید'
  # prompt = 'پاسخ پرسش'+ '"' + qa['question'] + '"' + 'بر اساس متن ترجمه مکارم شیرازی از آیات قرآن چیست؟ فقط پاسخ را بدون جمله کامل ارائه دهید'
  # print(prompt)
  response = openai.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=50,
        n=1
  )
  qa['gpt'] = response.choices[0].text
  new_list2.append(qa)
  # print(qa['gpt'])
  # break

with open('/content/drive/MyDrive/chatgpt_non_referential_part2.json', 'w', encoding = 'utf-8-sig') as f:
  json.dump(new_list2, f, ensure_ascii = False)

## test

In [ ]:
from evaluate import load
import json
from parsivar import Normalizer

my_normalizer = Normalizer()

with open('/content/drive/MyDrive/chatgpt_non_referential_part1.json', 'r', encoding = 'utf-8-sig') as f:
  data_list1 = json.load(f)

with open('/content/drive/MyDrive/chatgpt_non_referential_part2.json', 'r', encoding = 'utf-8-sig') as f:
  data_list2 = json.load(f)

data_list = data_list1 + data_list2

predictions = []
references = []

for data in data_list:
  if data['answers'][0]['text'] in data['gpt']:
    # predictions.append(data['answers'][0]['text'])
    # references.append(data['answers'][0]['text'])
    predictions.append(my_normalizer.normalize(data['answers'][0]['text']))
    references.append(my_normalizer.normalize(data['answers'][0]['text']))
  else:
    # predictions.append(data['gpt'].strip())
    # references.append(data['answers'][0]['text'])
    predictions.append(my_normalizer.normalize(data['gpt'].strip()))
    references.append(my_normalizer.normalize(data['answers'][0]['text']))

exact_match_metric = load("exact_match")
results = exact_match_metric.compute(predictions=predictions, references=references)

results

{'exact_match': 0.06794013391098858}

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

# f1 = f1_score(ground_truth, predicted_output, average='micro')
f1 = f1_score(references, predictions, average='weighted')

f1

0.10486502786647738

In [ ]:
def ask_chat_gpt(question):

    #Generate a response from ChatGPT using the question chat message
    response = openai.ChatCompletion.create(
    model = model,
    messages = [
        {"role": "user", "content": question}
    ],
    max_tokens=100,
    temperature=0.9,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    )

    #Extract the answer from the response
    answer = response.choices[0].message['content']

    #Return the answer
    return answer

In [ ]:
prompt = 'پاسخ پرسش'+ '"' + referral[0]['question'] + '"' + 'بر اساس متن ترجمه مکارم شیرازی از آیات قرآن چیست؟ فقط پاسخ را بدون جمله کامل ارائه دهید'
  # print(prompt)
response = openai.Completion.create(
      model="gpt-3.5-turbo-instruct",
      prompt=prompt,
      max_tokens=50,
      n=1,
)
print(response)

{
  "id": "cmpl-9SVMz609KRymt1puG7db0gTvwGdfo",
  "object": "text_completion",
  "created": 1716580797,
  "model": "gpt-3.5-turbo-instruct",
  "choices": [
    {
      "text": "\"\n\n\"\u0634\u0645\u0633\"",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 114,
    "completion_tokens": 7,
    "total_tokens": 121
  }
}


In [ ]:
generated_text = response.choices[0].text
print(generated_text)

"

"شمس"


In [ ]:
referral[1]

{'question': 'در آیه 113 سوره توبه به کدام دسته از انسان\u200cها اشاره شده است؟',
 'pq_id': 's9.113',
 'context': 'برای پیامبر و مؤمنان، شایسته نبود که برای مشرکان طلب آمرزش کنند، هر چند از نزدیکانشان باشند؛ پس از آنکه بر آنها روشن شد که این گروه، اهل دوزخند!',
 'answers': [{'text': 'مؤمنان', 'start_char': 14}]}

In [ ]:
!pip install -q OpenAI

In [ ]:
def main():
    """
    Main interaction loop for the chatbot.
    """
    print("Welcome to Chatbot! Type 'quit' to exit.")

    user_input = ""
    while user_input.lower() != "quit":
        user_input = input("You: ")

        if user_input.lower() != "quit":
            response = chat_with_openai(user_input)  # Pass user_input as an argument
            print(f"Chatbot: {response}")

In [ ]:
def chat_with_openai(prompt):
    """
    Sends the prompt to OpenAI API using the chat interface and gets the model's response.
    """
    message = {
        'role': 'user',
        'content': prompt
    }

    response = openai.chat.completions.create(
        model=model_name,
        messages=[message]
    )

    # Extract the chatbot's message from the response.
    # Assuming there's at least one response and taking the last one as the chatbot's reply.
    chatbot_response = response.choices[0].message['content']
    return chatbot_response.strip()


if __name__ == "__main__":
    main()

Welcome to Chatbot! Type 'quit' to exit.
You: 


AttributeError: module 'openai' has no attribute 'chat'

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "user", "content": "Who won the world series in 2020?"}
  ]
)
print(response.choices[0].message.content)

ImportError: cannot import name 'OpenAI' from 'openai' (/usr/local/lib/python3.10/dist-packages/openai/__init__.py)

save referential and non-referential

In [ ]:
import json

with open('/content/drive/MyDrive/train_v9.json', 'r', encoding = 'utf-8-sig') as f:
  train_list = json.load(f)

with open('/content/drive/MyDrive/dev_v9.json', 'r', encoding = 'utf-8-sig') as f:
  dev_list = json.load(f)

total_data = train_list + dev_list

non_referral = []
referral = []

str1 = 'سوره'
str2 = 'نام ببرید'
str3 = 'یکی از انواع'

for qa in total_data:
  question = qa['question']
  if str1 not in question and str2 not in question and str3 not in question:
    non_referral.append(qa)
  else:
    referral.append(qa)

In [ ]:
for i in range(len(referral)):
  prompt = 'پاسخ پرسش'+ '"' + referral[i]['question'] + '"' + 'بر اساس متن ترجمه مکارم شیرازی از آیات قرآن چیست؟ فقط پاسخ را بدون جمله کامل ارائه دهید'
  referral[i]['prompt'] = prompt

with open('content/drive/MyDrive/referential.json', 'w', encoding = 'utf-8-sig') as f:
  f.dump(referral, f, ensure_ac)

In [ ]:
str = "\n\n ..................................\nموسی از ظرف راست کوه طور فراخوانی شد.\n"

str2 = my_normalizer.normalize(str.strip())
str2

'..................................\n موسی از ظرف راست کوه طور فراخوانی شد .'